In [2]:
import pandas as pd

In [3]:
results = pd.read_parquet('../data/raw/game_results.parquet')

In [7]:
results

,season,round,venue,date,home_team,home_team_points,away_team,away_team_points
0,2018,25,"Stadium Australia, Sydney",2018-08-30,Rabbitohs,51,Wests Tigers,10
1,2018,25,"Mt Smart Stadium, Auckland",2018-08-31,Warriors,20,Raiders,16
2,2018,25,"AAMI Park, Melbourne",2018-08-31,Storm,16,Panthers,22
3,2018,25,"McDonald Jones Stadium, Newcastle",2018-09-01,Knights,14,Dragons,24
4,2018,25,"Cbus Super Stadium, Gold Coast",2018-09-01,Titans,26,Cowboys,30
...,...,...,...,...,...,...,...,...
3,2023,1,"4 Pines Park, Sydney",2023-03-04,Sea Eagles,None,Bulldogs,None
4,2023,1,"Queensland Country Bank Stadium, Townsville",2023-03-04,Cowboys,None,Raiders,None
5,2023,1,"PointsBet Stadium, Sydney",2023-03-04,Sharks,None,Rabbitohs,None
6,2023,1,"Suncorp Stadium, Brisbane",2023-03-05,Dolphins,None,Roosters,None


In [12]:

def result_expander(rdf):
    '''
    Duplicate results table such that each game has a row where each team is prime and opt.

    This creates a dataframe that can be used for creating lagged effects on each team's performance.
    '''
    results_prime = rdf.copy()
    results_opt = rdf.copy()
    prime_cols = ['season', 'round', 'venue', 'date', 'prime_team', 'prime_team_points',
                             'opt_team', 'opt_team_points']
    opt_cols = ['season', 'round', 'venue', 'date', 'opt_team', 'opt_team_points',
                            'prime_team', 'prime_team_points']
    results_prime.columns = prime_cols
    results_opt.columns = opt_cols

    all_results = pd.concat([results_prime.loc[:, prime_cols], results_opt.loc[:, prime_cols]])

    return all_results

    


In [23]:
expanded_results = result_expander(results)
expanded_results = expanded_results.assign(prime_team_points=expanded_results.prime_team_points.astype('float'))
expanded_results = expanded_results.assign(opt_team_points=expanded_results.opt_team_points.astype('float'))
expanded_results = expanded_results.assign(prime_margin=expanded_results.prime_team_points - expanded_results.opt_team_points)
expanded_results = expanded_results.assign()

In [26]:
expanded_results[(expanded_results.prime_team == 'Cowboys') & 
                ~(expanded_results.prime_team_points.isna())].sort_values('date').shift(1)

,season,round,venue,date,prime_team,prime_team_points,opt_team,opt_team_points,margin
2,NaN,NaN,None,None,None,NaN,None,NaN,NaN
2,2018.0,1.0,"1300SMILES Stadium, Townsville",2018-03-09,Cowboys,20.0,Sharks,14.0,6.0
0,2018.0,2.0,"Suncorp Stadium, Brisbane",2018-03-16,Cowboys,20.0,Broncos,24.0,-4.0
0,2018.0,3.0,"AAMI Park, Melbourne",2018-03-22,Cowboys,14.0,Storm,30.0,-16.0
4,2018.0,4.0,"1300SMILES Stadium, Townsville",2018-03-29,Cowboys,14.0,Panthers,33.0,-19.0
...,...,...,...,...,...,...,...,...,...
6,2022.0,20.0,"Netstrata Jubilee Stadium, Sydney",2022-07-31,Cowboys,34.0,Dragons,8.0,26.0
3,2022.0,21.0,"Salter Oval, Bundaberg",2022-08-07,Cowboys,28.0,Bulldogs,14.0,14.0
1,2022.0,22.0,"Sydney Cricket Ground, Sydney",2022-08-13,Cowboys,18.0,Roosters,32.0,-14.0
5,2022.0,23.0,"Queensland Country Bank Stadium, Townsville",2022-08-19,Cowboys,48.0,Warriors,4.0,44.0


In [ ]:
def features(team_result_df):
    
    team_result_df = team_result_df.sort_values('date')
    lagged_team_result_df = team_result_df.copy()